In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install protobuf==4.25.3 --force-reinstall


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.38.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible

In [4]:
# %% [markdown]
# # 🧠 Pega Rules → SQLite → Vector Database (Chroma)
# 
# This notebook reads your Pega rule data from SQLite, converts it into
# embeddings, stores it in a Chroma vector database, and allows semantic search.
# 
# Optional next step: integrate Mistral (4-bit) for reasoning.

# %%
# 1️⃣ Install dependencies
!pip install -q chromadb sentence-transformers sqlite-utils


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:

# %%
# 2️⃣ Imports
import sqlite3
import json
from chromadb import Client
from sentence_transformers import SentenceTransformer

# %%
# 3️⃣ Setup: database + embedding model
db_path = "/kaggle/input/rulesdb/rules.db"  # 🔁 update this

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize Chroma vector DB
chroma = Client()
collection = chroma.get_or_create_collection("pega_rules_db")


In [6]:

# %%
# 4️⃣ Helper function to convert each rule row into meaningful text
def format_rule_text(row):
    """
    Convert DB row to descriptive text for semantic embedding.
    row order: (id, key, rule_type, rule_class, name, description, rulejson, businessdescription)
    """
    try:
        rule_json = json.loads(row[6]) if row[6] else {}
    except Exception:
        rule_json = {}

    return f"""
Rule Key: {row[1]}
Rule Type: {row[2]}
Rule Class: {row[3]}
Rule Name: {row[4]}
Description: {row[5]}
Business Description: {row[7]}
Rule JSON Summary: {json.dumps(rule_json, indent=2)[:1000]}
"""


In [15]:

# %%
# 5️⃣ Fetch rules and insert into the vector DB
cursor.execute("SELECT * FROM top_rules")
rows = cursor.fetchall()

for row in rows:
    rule_id = str(row[0])
    text = format_rule_text(row)
    embedding = embedder.encode(text).tolist()

    collection.add(
        ids=[rule_id],
        embeddings=[embedding],
        metadatas=[{
            "key": row[1],
            "rule_type": row[2],
            "rule_class": row[3],
            "name": row[4]
        }],
        documents=[text]
    )

print(f"✅ Loaded {len(rows)} rules into vector database.")


✅ Loaded 20266 rules into vector database.


In [14]:

# %%
# 6️⃣ Example query: find relevant rules for a user story
user_story = "We need to validate email address in the customer registration form."

query_embedding = embedder.encode(user_story).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=5)

print("\n🔍 Top Matching Rules:\n")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"{meta['name']} ({meta['rule_type']}) - {meta['rule_class']}")
    print(doc[:400], "\n---\n")



# %%
# ✅ Summary
# You now have:
# - Pega rules indexed from SQLite
# - A working vector database for semantic retrieval
# - Ready integration point for a reasoning model like Mistral 4-bit

# Next step (optional):
# - Integrate Mistral to interpret user stories and explain rule changes.
# Would you like me to add that part next?



🔍 Top Matching Rules:

ValUAUserInformation (Rule-Obj-Model) - Code-Security

Rule Key: RULE-OBJ-MODEL CODE-SECURITY VALUAUSERINFORMATION #20190426T065753.023 GMT
Rule Type: Rule-Obj-Model
Rule Class: Code-Security
Rule Name: ValUAUserInformation
Description: validates the unauthentication user information.
Business Description: 

This rule retrieves the operator information based on the provided UserIdentifier. If the UserIdentifier is empty, it sets the status message to 
---

PopulateFromEmailStringHeaderExt (Rule-Obj-Model) - Data-EmailAccount

Rule Key: RULE-OBJ-MODEL DATA-EMAILACCOUNT POPULATEFROMEMAILSTRINGHEADEREXT #20230306T084433.108 GMT
Rule Type: Rule-Obj-Model
Rule Class: Data-EmailAccount
Rule Name: PopulateFromEmailStringHeaderExt
Description: Impl teams who wants to send the email on users email instead of system account email address
Business Description: 

This rule is designed to set the email address of a superclass obje 
---

GetEmailAccountName (Rule-Obj-Model) -

In [ ]:
# %%
# 7️⃣ (Optional) Persist the Chroma DB for reuse
collection.persist()
print("💾 Vector database saved successfully.")